# KAYAK - DATA INFRASTRUCTURE PIPELINE

## Business Context

Kayak is a travel search engine aiming to help users plan their next holidays based on reliable and objective data.

User research conducted by the Marketing team highlighted that:
- 70% of users would like more trusted information about destinations before booking
- Users tend to distrust content if the data source is unknown

Therefore, the Marketing team wants to build a recommendation application able to suggest:

- The best destinations to travel to
- The best hotels available in those destinations

based on real-time environmental conditions such as:

- Weather forecast
- Accommodation quality

To support this application, a data infrastructure must be implemented in order to collect, store, process and expose reliable travel-related data.

This notebook presents the implementation of a Data Engineering pipeline designed to support this business need.

## Imports

In [1]:
import requests
import pandas as pd
import time

## Data Architecture Overview

In order to support Kayak’s future recommendation system, a data infrastructure has been designed following a modern Data Lake / Data Warehouse architecture:

### Data Sources
- Geolocation API (Nominatim)
- Weather API (Open-Meteo)
- Hotel information (Booking.com scraping)

### Data Lake (AWS S3)
Raw and enriched data collected from external sources are stored in a centralized storage layer (AWS S3) acting as a Data Lake.

### ETL Pipeline
An Extract – Transform – Load process is implemented to:
- Clean collected hotel data
- Standardize review scores
- Handle missing values
- Compute a climate attractiveness indicator

### Data Warehouse (AWS RDS PostgreSQL)
Cleaned and enriched data are then loaded into a structured relational database in order to:
- Facilitate downstream analysis
- Enable marketing teams to query trusted travel insights

## Destinations Scope Definition

In [2]:
CITIES = [
    "Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen",
    "Paris", "Amiens", "Lille", "Strasbourg",
    "Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim",
    "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon",
    "Gorges du Verdon", "Bormes les Mimosas", "Cassis", "Marseille",
    "Aix en Provence", "Avignon", "Uzes", "Nimes", "Aigues Mortes",
    "Saintes Maries de la mer", "Collioure", "Carcassonne",
    "Ariege", "Toulouse", "Montauban", "Biarritz", "Bayonne",
    "La Rochelle"
]

NOMINATIM_API_URL = "https://nominatim.openstreetmap.org/search"
HEADERS = {"User-Agent": "kayak-data-team"}

## GPS Coordinates Collection

In [3]:
gps_data=[]

for idx, city in enumerate(CITIES, start=1):

    params = {
        "q": f"{city}, France",
        "format": "json",
        "limit":1
    }

    response = requests.get(
        NOMINATIM_API_URL,
        headers=HEADERS,
        params=params
    )

    time.sleep(1)

    if response.status_code==200 and response.json():
        lat = response.json()[0]["lat"]
        lon = response.json()[0]["lon"]
    else:
        lat, lon = None, None

    gps_data.append({
        "city_id": idx,
        "city_name": city,
        "latitude": lat,
        "longitude": lon,
    })        
cities_df= pd.DataFrame(gps_data)
cities_df    

,city_id,city_name,latitude,longitude
0,1,Mont Saint Michel,48.6359541,-1.5114600
1,2,St Malo,48.6495180,-2.0260409
2,3,Bayeux,49.2764624,-0.7024738
3,4,Le Havre,49.4938975,0.1079732
4,5,Rouen,49.4404591,1.0939658
5,6,Paris,48.8534951,2.3483915
6,7,Amiens,49.8941708,2.2956951
7,8,Lille,50.6365654,3.0635282
8,9,Strasbourg,48.5846140,7.7507127
9,10,Chateau du Haut Koenigsbourg,48.2493820,7.3439412


## Weather Data Collection

In [4]:
OPENMETEO_API_URL = "https://api.open-meteo.com/v1/forecast"

In [5]:
weather_data = []

for _, row in cities_df.iterrows():

    params = {
        "latitude": row["latitude"],
        "longitude": row["longitude"],
        "daily": "temperature_2m_max,precipitation_sum",
        "forecast_days": 7,
        "timezone": "auto"
    }

    response = requests.get(OPENMETEO_API_URL, params=params)

    if response.status_code == 200:

        data = response.json()

        rain = sum(data["daily"]["precipitation_sum"])
        temp = sum(data["daily"]["temperature_2m_max"]) / 7

        weather_data.append({
            "city_id": row["city_id"],
            "city_name": row["city_name"],
            "rain_7d": rain,
            "avg_temp_7d": temp
        })

    else:
        print(f"Error for {row['city_name']} : {response.status_code}")    

print(len(weather_data))

weather_df = pd.DataFrame(weather_data)
weather_df


35


,city_id,city_name,rain_7d,avg_temp_7d
0,1,Mont Saint Michel,8.48,13.300000
1,2,St Malo,7.58,13.542857
2,3,Bayeux,13.50,13.542857
3,4,Le Havre,16.70,12.500000
4,5,Rouen,18.60,13.785714
5,6,Paris,12.35,14.942857
6,7,Amiens,9.90,14.128571
7,8,Lille,13.60,13.314286
8,9,Strasbourg,9.30,14.242857
9,10,Chateau du Haut Koenigsbourg,5.96,11.357143


## Weather Attractiveness Index Engineering

In order to recommend destinations based on short-term travel comfort,
a composite indicator named **Weather Score** has been engineered.

This score aims to quantify the climatic attractiveness of each destination
over the next 7 days by combining:

- Average daily maximum temperature
- Expected cumulative precipitation

The objective of this index is to prioritize destinations offering:
- Warmer temperatures
- Lower rainfall levels

This indicator will later be used by the Marketing team to identify
the most suitable destinations for holiday planning.

## Weather Score Calculation

In [6]:
weather_df["weather_score"] = (
    weather_df["avg_temp_7d"] - (weather_df["rain_7d"] * 0.5)
)

top_destinations = weather_df.sort_values(
    by="weather_score",
    ascending=False
).head(5)

top_destinations

,city_id,city_name,rain_7d,avg_temp_7d,weather_score
15,16,Grenoble,0.0,17.671429,17.671429
22,23,Avignon,0.0,17.528571,17.528571
24,25,Nimes,0.0,17.385714,17.385714
23,24,Uzes,0.0,16.871429,16.871429
20,21,Marseille,0.0,16.842857,16.842857


## First Consolidation

In [7]:
cities_weather_df = cities_df.merge(
    weather_df,
    on=["city_id", "city_name"],
    how="inner"
)

cities_weather_df

,city_id,city_name,latitude,longitude,rain_7d,avg_temp_7d,weather_score
0,1,Mont Saint Michel,48.6359541,-1.5114600,8.48,13.300000,9.060000
1,2,St Malo,48.6495180,-2.0260409,7.58,13.542857,9.752857
2,3,Bayeux,49.2764624,-0.7024738,13.50,13.542857,6.792857
3,4,Le Havre,49.4938975,0.1079732,16.70,12.500000,4.150000
4,5,Rouen,49.4404591,1.0939658,18.60,13.785714,4.485714
5,6,Paris,48.8534951,2.3483915,12.35,14.942857,8.767857
6,7,Amiens,49.8941708,2.2956951,9.90,14.128571,9.178571
7,8,Lille,50.6365654,3.0635282,13.60,13.314286,6.514286
8,9,Strasbourg,48.5846140,7.7507127,9.30,14.242857,9.592857
9,10,Chateau du Haut Koenigsbourg,48.2493820,7.3439412,5.96,11.357143,8.377143


In [8]:
import plotly.express as px

In [9]:
top_destinations_map = top_destinations.merge(
    cities_weather_df[["city_id", "latitude", "longitude"]],
    on="city_id",
    how="inner"
)

top_destinations_map["latitude"] = pd.to_numeric(top_destinations_map["latitude"], errors="coerce")
top_destinations_map["longitude"] = pd.to_numeric(top_destinations_map["longitude"], errors="coerce")

top_destinations_map


,city_id,city_name,rain_7d,avg_temp_7d,weather_score,latitude,longitude
0,16,Grenoble,0.0,17.671429,17.671429,45.187560,5.735782
1,23,Avignon,0.0,17.528571,17.528571,43.949249,4.805901
2,25,Nimes,0.0,17.385714,17.385714,43.837425,4.360069
3,24,Uzes,0.0,16.871429,16.871429,44.012128,4.419672
4,21,Marseille,0.0,16.842857,16.842857,43.296174,5.369953


In [10]:
fig = px.scatter_map(
    top_destinations_map,
    lat="latitude",
    lon="longitude",
    size="weather_score",
    color="weather_score",
    hover_name="city_name",
    hover_data=["rain_7d", "avg_temp_7d"],
    zoom=5,
    height=600
)

fig.show()


## Hotel Information Collection (Web Scraping)

As Booking Holdings does not provide aggregated public APIs for hotel data,
hotel information has been collected directly from Booking.com through automated web scraping.

The following attributes are retrieved for each hotel:
- Hotel name
- User review score
- Booking page URL
- Accommodation description

This data will allow Kayak’s future recommendation system to:
- Identify high-quality accommodation options
- Associate hotel quality with destination attractiveness

## Hotels Data Collection

In [22]:
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.common.by import By
from selenium import webdriver
import time
import random

In [23]:
options = FirefoxOptions()

options.add_argument("--width=1920")
options.add_argument("--height=1080")

driver = webdriver.Firefox(
    service=FirefoxService(GeckoDriverManager().install()),
    options=options
)

In [24]:
driver.get("https://www.booking.com")

time.sleep(5)

In [25]:
hotels_data = []

for _, row in top_destinations.iterrows():

    city = row["city_name"]

    url = f"https://www.booking.com/searchresults.html?ss={city}"

    driver.get(url)

    time.sleep(random.uniform(5,7))

    hotels = driver.find_elements(By.CSS_SELECTOR,'div[data-testid="property-card"]')[:20]

    print(f"{city} -> {len(hotels)} hotels found")

    for hotel in hotels:

        try:
            name = hotel.find_element(By.CSS_SELECTOR,'div[data-testid="title"]').text
        except:
            name = None

        try:
            score = hotel.find_element(By.CSS_SELECTOR,'div[data-testid="review-score"]').text
        except:
            score = None

        try:
            description = hotel.find_element(By.CSS_SELECTOR,'div[data-testid="property-card-unit-configuration"]').text
        except:
            description = None

        try:
            link = hotel.find_element(By.TAG_NAME,"a").get_attribute("href")
        except:
            link = None

        hotels_data.append({
            "city_id": row["city_id"],
            "city_name": city,
            "hotel_name": name,
            "review_score": score,
            "description": description,
            "booking_url": link
        })

    time.sleep(random.uniform(3,5))

hotels_df = pd.DataFrame(hotels_data)

driver.quit()

Grenoble -> 20 hotels found
Avignon -> 20 hotels found
Nimes -> 20 hotels found
Uzes -> 20 hotels found
Marseille -> 20 hotels found


In [32]:
hotels_clean_df = hotels_df.copy()

hotels_clean_df["review_score"] = (
    hotels_clean_df["review_score"]
    .astype(str)
    .str.replace(",", ".", regex=False)
    .str.extract(r"(\d+\.?\d*)")
    .astype(float)
)

hotels_clean_df = hotels_clean_df.fillna({
    "hotel_name": "Not Available",
    "review_score": 0,
    "booking_url": "Not Available"
})

hotels_clean_df

,city_id,city_name,hotel_name,review_score,description,booking_url
0,16,Grenoble,Le Berthelot - Terrasse avec Vue Montagne,0.0,None,https://www.booking.com/hotel/fr/le-berthelot-...
1,16,Grenoble,Apparthôtel Tempologis - St Germain à Grenoble,8.3,None,https://www.booking.com/hotel/fr/tempologis-gr...
2,16,Grenoble,Appartement Loft centre ville situé dans rue c...,9.5,None,https://www.booking.com/hotel/fr/appartement-l...
3,16,Grenoble,Villa atmosphère PETIT DEJEUNER COMPRIS,9.3,None,https://www.booking.com/hotel/fr/sarl-residila...
4,16,Grenoble,Good Vibes,9.2,None,https://www.booking.com/hotel/fr/good-vibes-gr...
...,...,...,...,...,...,...
95,21,Marseille,RockyPop Marseille Hôtel,9.1,None,https://www.booking.com/hotel/fr/citadines-apa...
96,21,Marseille,Novotel Marseille Vieux Port,7.8,None,https://www.booking.com/hotel/fr/novotel-marse...
97,21,Marseille,HOTEL SYLVABELLE,4.7,None,https://www.booking.com/hotel/fr/sylvabelle-ma...
98,21,Marseille,"appartement renové , quartier historique du pa...",7.6,None,https://www.booking.com/hotel/fr/appart-renove...


## 🔄 ETL - Data Transformation & Consolidation

Once collected, weather and hotel datasets are merged
to create an enriched travel dataset.

Transformation steps include:
- Cleaning of review score values
- Standardization of numeric formats
- Handling missing values

The resulting dataset combines:
- Geographical data
- Weather indicators
- Accommodation quality metrics

This consolidated dataset represents a curated layer
ready to be stored within the Data Lake.

## Final Consolidation

In [33]:
enriched_travel_df = cities_weather_df.merge(
    hotels_clean_df,
    on=["city_id", "city_name"],
    how="left"
)

enriched_travel_df

,city_id,city_name,latitude,longitude,rain_7d,avg_temp_7d,weather_score,hotel_name,review_score,description,booking_url
0,1,Mont Saint Michel,48.6359541,-1.5114600,8.48,13.300000,9.060000,NaN,NaN,NaN,NaN
1,2,St Malo,48.6495180,-2.0260409,7.58,13.542857,9.752857,NaN,NaN,NaN,NaN
2,3,Bayeux,49.2764624,-0.7024738,13.50,13.542857,6.792857,NaN,NaN,NaN,NaN
3,4,Le Havre,49.4938975,0.1079732,16.70,12.500000,4.150000,NaN,NaN,NaN,NaN
4,5,Rouen,49.4404591,1.0939658,18.60,13.785714,4.485714,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
125,31,Toulouse,43.6044638,1.4442433,2.40,16.528571,15.328571,NaN,NaN,NaN,NaN
126,32,Montauban,44.0175835,1.3549991,2.40,16.442857,15.242857,NaN,NaN,NaN,NaN
127,33,Biarritz,43.4832523,-1.5592776,0.60,16.657143,16.357143,NaN,NaN,NaN,NaN
128,34,Bayonne,43.4945144,-1.4736657,0.60,17.085714,16.785714,NaN,NaN,NaN,NaN


In [34]:
enriched_travel_df.to_csv("enriched_travel_data.csv", index=False)

## Data Lake Storage (AWS S3)

The enriched dataset is uploaded to an AWS S3 bucket,
which acts as a centralized Data Lake storage layer.

This layer enables:
- Persistent storage of collected external data
- Decoupling between ingestion and analytics environments
- Future scalability for additional travel-related datasets

## Cloud Storage - AWS S3

In [35]:
import boto3
import os

In [36]:
S3_BUCKET_NAME = "kayak-data-lake-bucket"
S3_FILE_NAME = "enriched_travel_data.csv"
S3_OBJECT_PATH = "curated/enriched_travel_data.csv"

In [37]:
from dotenv import load_dotenv
load_dotenv()

AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_REGION = os.getenv("AWS_DEFAULT_REGION")

In [38]:
s3_client = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION
)

In [39]:

s3_client.upload_file(
    Filename=S3_FILE_NAME,
    Bucket=S3_BUCKET_NAME,
    Key=S3_OBJECT_PATH,
)

## Data Warehouse Loading (AWS RDS)

To support downstream analytics and marketing decision-making,
data stored in the Data Lake are extracted and loaded
into a structured PostgreSQL database hosted on AWS RDS.

This Data Warehouse provides:
- Structured access to travel insights
- Queryable datasets for business intelligence use cases
- Reliable inputs for Kayak’s recommendation application

## Data Warehouse - AWS RDS

In [40]:
from sqlalchemy import create_engine

### Data Extraction from Data Lake

In [41]:
s3_client.download_file(
    Bucket=S3_BUCKET_NAME,
    Key=S3_OBJECT_PATH,
    Filename=S3_FILE_NAME
)

### Data Transformation

In [42]:
extracted_travel_df  = pd.read_csv(S3_FILE_NAME)

### RDS Configuration

In [43]:
RDS_HOST = os.getenv("RDS_HOST")
RDS_PORT = os.getenv("RDS_PORT")
RDS_DB = os.getenv("RDS_DB_NAME")
RDS_USER = os.getenv("RDS_USER")
RDS_PASSWORD = os.getenv("RDS_PASSWORD")

### Database Connection

In [44]:
engine = create_engine(
    f"postgresql://{RDS_USER}:{RDS_PASSWORD}@{RDS_HOST}:{RDS_PORT}/{RDS_DB}"
)

### Data Loading into Warehouse

In [45]:
extracted_travel_df.to_sql(
    name="travel_data",
    con=engine,
    if_exists="replace",
    index=False
)

130

### Data Verification

In [46]:
pd.read_sql("""
SELECT city_name, hotel_name, review_score
FROM travel_data
ORDER BY review_score DESC
LIMIT 10
""", engine)

,city_name,hotel_name,review_score
0,St Malo,None,None
1,Bayeux,None,None
2,Le Havre,None,None
3,Rouen,None,None
4,Paris,None,None
5,Amiens,None,None
6,Lille,None,None
7,Strasbourg,None,None
8,Chateau du Haut Koenigsbourg,None,None
9,Mont Saint Michel,None,None


## Decision-Support Dataset

The structured dataset stored in the Data Warehouse
can now be queried to:

- Identify the most attractive destinations
- Highlight top-rated hotels within those locations

This enables Kayak’s Marketing team to:
- Recommend destinations based on climate conditions
- Suggest high-quality accommodation options

## Top 20 Hotels Selection

In [48]:
top_20_hotels = extracted_travel_df.sort_values(
    by="review_score",
    ascending=False
).head(20)

top_20_hotels

,city_id,city_name,latitude,longitude,rain_7d,avg_temp_7d,weather_score,hotel_name,review_score,description,booking_url
66,23,Avignon,43.949249,4.805901,0.0,17.528571,17.528571,Maison climatisée intra-muros avec terrasse au...,10.0,NaN,https://www.booking.com/hotel/fr/maison-famili...
60,23,Avignon,43.949249,4.805901,0.0,17.528571,17.528571,Grande maison de ville intra-muros - Calme & c...,10.0,NaN,https://www.booking.com/hotel/fr/grande-maison...
78,23,Avignon,43.949249,4.805901,0.0,17.528571,17.528571,Le Complot : Maison privée en plein coeur d'Av...,9.8,NaN,https://www.booking.com/hotel/fr/le-complot-ma...
82,24,Uzes,44.012128,4.419672,0.0,16.871429,16.871429,Le Pré Aux Cigales,9.8,NaN,https://www.booking.com/hotel/fr/le-pre-aux-ci...
97,24,Uzes,44.012128,4.419672,0.0,16.871429,16.871429,Secret d'Uzès - Le Duché - Piscine chauffée et...,9.8,NaN,https://www.booking.com/hotel/fr/secret-d-uzes...
116,25,Nimes,43.837425,4.360069,0.0,17.385714,17.385714,"AfroBoHome, Expérience Atypique, clim, 2 à 6 p...",9.8,NaN,https://www.booking.com/hotel/fr/afrobohome-te...
85,24,Uzes,44.012128,4.419672,0.0,16.871429,16.871429,La Grande Bourgade-Authentique Maison en Pierr...,9.7,NaN,https://www.booking.com/hotel/fr/la-grande-bou...
74,23,Avignon,43.949249,4.805901,0.0,17.528571,17.528571,Chez Flo,9.7,NaN,https://www.booking.com/hotel/fr/chez-flo-avig...
92,24,Uzes,44.012128,4.419672,0.0,16.871429,16.871429,La bohème place aux herbes,9.7,NaN,https://www.booking.com/hotel/fr/la-boheme-uze...
112,25,Nimes,43.837425,4.360069,0.0,17.385714,17.385714,Roma Divine home cinéma et jardin,9.7,NaN,https://www.booking.com/hotel/fr/roma-divine-h...


In [49]:
import plotly.express as px

In [50]:
fig = px.scatter_map(
    top_20_hotels,
    lat="latitude",
    lon="longitude",
    size="review_score",
    color="review_score",
    hover_name="hotel_name",
    hover_data=["city_name"],
    zoom=5,
    height=600
)

fig.show()
